In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import glob
import imgkit
import dataframe_image as dfi

In [2]:
ptm_df = pd.read_csv('Data/Conserved_ptms_best_loc.csv')
mod_residues_dic = []
for i in ptm_df.mod_residues_dic:
    d = ast.literal_eval(i)
    mod_residues_dic.append(d)
ptm_df.mod_residues_dic = mod_residues_dic
num_species = []
for index, row in ptm_df.iterrows():
    d = row['mod_residues_dic']
    species = 0
    for key in d:
        if len(d[key][0]) > species:
            species = len(d[key][0])
    num_species.append(species)
ptm_df['num_species'] = num_species
ptm_df = ptm_df.sort_values('num_species', ascending = False)
ptm_df = ptm_df[ptm_df.num_species >=5]
residues = ['ammonium', 'Methyl:2H(3)13C(1)', 'Delta:H(4)C(2)O(-1)S(1)', 'Fluoro', 'Cation:Mg[II]', 'Acetyl:13C(2)', 'Label:2H(3)+Oxidation', 'Cation:Na', 'Cation:Fe[III]', 'Cation:Ca[II]', 'Label:13C(4)', 'Methyl:2H(3)']
ptm_df = ptm_df[~ptm_df.Modification.isin(residues)]

In [3]:
pd.unique(ptm_df[ptm_df.Gene == 'tuf'].Modification)

array(['Ammonium', 'Carbamyl', 'Ammonia-loss', 'Dehydrated', 'Carboxy',
       'Methyl', 'glyoxalAGE', 'Oxidation'], dtype=object)

In [4]:
def parse_meg(file):
    rows = []
    file = open(file)
    fasta_id = ''
    seq = str()
    i = 0
    d = dict()
    for line in file:
        
        if '#' in line and 'mega' not in line:
            if len(seq) > 0 and len(fasta_id) > 0:
                d['fasta_id'] = fasta_id
                d['sequence'] = seq
                rows.append(d)
                i += 1
            line = line.replace('\n', '').replace('#', '')
            fasta_id = line.strip()
            seq = ''
            d = dict()
        else:
            seq = seq + line.replace('\n', '') 
    d['fasta_id'] = fasta_id
    d['sequence'] = seq
    rows.append(d)
    
    return pd.DataFrame(rows)

In [5]:
files = glob.glob('Data/Alignments/*.meg')
meg_files = []
for file in files:
    meg = parse_meg(file)
    meg_files.append(meg)
meg_df = pd.concat(meg_files)

In [6]:
def styling_specific_cell(x,row_idx,col_idx, cellcolor, txtcolor = 'white'):
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    for row, col, ccolor in zip(row_idx, col_idx, cellcolor):
        color = 'background-color: {}; color: {}'.format(ccolor,txtcolor)
        df_styler.loc[row, col] = color
    return df_styler

In [7]:
def color_alignment(gene_name, vert = False):
    gene_meg = meg_df[meg_df.fasta_id.str.contains(gene_name)]
    gene_ptm = ptm_df[ptm_df.Gene == gene_name]
    rows = []
    for index, row in gene_meg.iterrows():
        new_row = dict()
        seq = row.sequence
        fasta_id = row.fasta_id
        new_row['Organism(gene)'] = fasta_id
        for i,c in zip(range(len(seq)),seq):
            new_row[i] = c
        rows.append(new_row)
    split_gene_meg = pd.DataFrame(rows)
    colors = ['red', 'green', 'cyan', 'blue', 'magenta', 'purple', 'orange', 'pink', 'brown', 'lime', 'teal', 'navy', '#B31B1B', '#703642', '#FFB200', '#FFBCD9', '#013220']
    if not vert:
        split_gene_meg.index = split_gene_meg['Organism(gene)']
        split_gene_meg=split_gene_meg.drop(columns = ['Organism(gene)'])
        rows = []
        cols = []
        colors_used = []
        i = 0
        for index, row in gene_ptm.iterrows():
            ptm_map = row.mod_residues_dic
            for key, value in ptm_map.items():
                fasta_ids = value[1]
                for fasta_id in fasta_ids:
                    rows.append(fasta_id)
                    cols.append(key)
                    colors_used.append(colors[i])
            i += 1
        split_gene_meg.columns.name = 'Residue Position'
    else:
        organisms = list(split_gene_meg['Organism(gene)'])
        del split_gene_meg['Organism(gene)']
        split_gene_meg = split_gene_meg.transpose()
        split_gene_meg.columns = organisms
        split_gene_meg = split_gene_meg[organisms]
        #split_gene_meg.index.name = 'Residue Position'
        split_gene_meg.columns.name = 'Organism(gene)'
        rows = []
        cols = []
        colors_used = []
        i = 0
        for index, row in gene_ptm.iterrows():
            ptm_map = row.mod_residues_dic
            for key, value in ptm_map.items():
                fasta_ids = value[1]
                for fasta_id in fasta_ids:
                    rows.append(key)
                    cols.append(fasta_id)
                    colors_used.append(colors[i])
            i += 1
        fasta_id_map = dict()
        new_cols = []
        for i, fasta_id in zip(range(0, len(split_gene_meg.columns)), split_gene_meg.columns):
            fasta_id_map[fasta_id] = i
            new_cols.append(i)
        split_gene_meg.columns = new_cols
        new_cols = []
        for fasta_id in cols:
            new_cols.append(fasta_id_map[fasta_id])
        cols = new_cols
    return(split_gene_meg.style.apply(styling_specific_cell, row_idx = rows, col_idx = cols, cellcolor = colors_used, axis= None))

In [8]:
prfB = color_alignment('prfB')

In [9]:
drop_cols = list(range(0,270))
drop_cols.extend(list(range(281, 411)))

In [10]:
prfB_chunk = prfB.hide_columns(drop_cols)
prfB_chunk

Residue Position,270,271,272,273,274,275,276,277,278,279,280
Organism(gene),,,,,,,,,,,
Acidiphilium_cryptum(prfB),A,S,G,A,G,G,Q,H,V,N,K
Bacillus_cereus(prfB),A,S,G,A,G,G,Q,H,V,N,T
Bacillus_subtilis(prfB),A,S,G,A,G,G,Q,H,V,N,T
Bacteroides_fragilis(prfB),S,G,G,A,G,G,Q,N,V,N,K
Cellulophaga_baltica(prfB),S,S,G,A,G,G,Q,N,V,N,K
Cellvibrio_gilvus(prfB),S,S,G,P,G,G,Q,S,V,N,T
Chryseobacterium_indologenes(prfB),S,S,G,A,G,G,Q,N,V,N,K
Citrobacter_freundii(prfB),A,S,G,A,G,G,Q,H,V,N,R
Coprococcus_comes(prfB),S,S,G,A,G,G,Q,H,I,N,K


In [11]:
dfi.export(prfB_chunk, 'Figures/prfB.png', max_cols= -1)

In [12]:
tuf = color_alignment('tuf', True)
dfi.export(tuf, 'Figures/tuf_narrow.png', max_rows=-1)

In [13]:
tuf

In [14]:
tuf = color_alignment('tuf', False)
dfi.export(tuf, 'Figures/tuf.png', max_cols=-1)
tuf